In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


## 데이터

### 적재

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/BPR/philadelphia_rating_BPR.csv")
df

,newUserId,newBusinessId
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
332974,11668,7007
332975,11914,7008
332976,11941,7009
332977,11978,7010


In [ ]:
# 사용자 수 출력
num_users = df["newUserId"].nunique()
print("사용자 수:", num_users)

# 음식점 수 출력
num_businesses = df["newBusinessId"].nunique()
print("음식점 수:", num_businesses)

사용자 수: 12062
음식점 수: 7011


### 데이터셋 분리

In [ ]:
import pandas as pd

# 데이터셋 분리
# TrainSet, TestSet = Data[n:], Data[:n]
# 사용자별 고르게 분리
def split_train_test_data(df, n=3, user_col='newUserId', item_col='newBusinessId', output_dir='.'):
    train_data = []
    test_data = []

    i=0

    # 사용자별로 그룹화하여 처리
    for user_id, group in df.groupby(user_col):
        # 첫 번째부터 n번째 음식점 방문 기록을 test_data로 추가
        test_data.extend(group.iloc[:n].values.tolist())

        # n+1번째부터 마지막 음식점 방문 기록을 train_data로 추가
        train_data.extend(group.iloc[n:].values.tolist())

        if(i<3):
          print(f"User ID: {user_id}")
          print("Test Data:")
          print(test_data)
          print("Train Data:")
          print(train_data)
          print("\n")
          i+=1


    train_df = pd.DataFrame(train_data, columns=[user_col, item_col])
    test_df = pd.DataFrame(test_data, columns=[user_col, item_col])


    train_size = train_df.shape
    test_size = test_df.shape
    print("Train 데이터 크기:", train_size)
    print("Test 데이터 크기:", test_size)
    print("Test 데이터 크기 / Train 데이터 크기 비율:", test_size[0] / train_size[0]) # 전체 크기의 대략 10%

    # CSV 파일로 저장
    train_df.to_csv(f"{output_dir}/train.csv", index=False, header=False, sep='\t')
    test_df.to_csv(f"{output_dir}/test.csv", index=False, header=False, sep='\t')

    print("'train.csv'와 'test.csv' 파일이 생성되었습니다.")

split_train_test_data(df, n=3, output_dir="/content/drive/MyDrive/BPR")


User ID: 1
Test Data:
[[1, 1], [1, 2], [1, 3]]
Train Data:
[[1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [1, 18], [1, 19], [1, 20], [1, 21]]


User ID: 2
Test Data:
[[1, 1], [1, 2], [1, 3], [2, 1], [2, 22], [2, 23]]
Train Data:
[[1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [1, 18], [1, 19], [1, 20], [1, 21], [2, 24], [2, 25], [2, 26], [2, 27], [2, 28], [2, 29], [2, 30], [2, 31]]


User ID: 3
Test Data:
[[1, 1], [1, 2], [1, 3], [2, 1], [2, 22], [2, 23], [3, 1], [3, 23], [3, 32]]
Train Data:
[[1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [1, 18], [1, 19], [1, 20], [1, 21], [2, 24], [2, 25], [2, 26], [2, 27], [2, 28], [2, 29], [2, 30], [2, 31], [3, 33], [3, 34], [3, 35], [3, 36], [3, 37], [3, 38], [3, 39], [3, 40], [3, 41], [3, 42], [3, 43], [3, 44], [3, 45], [3, 46], 

### 사용자별 미방문(부정적) 아이템셋 생성

In [ ]:
import pandas as pd
import numpy as np
import random

# 각 사용자별로 부정적인 조합 생성
def generate_negative_samples(df, n=100, user_col='newUserId', item_col='newBusinessId', output_dir='.'):
    # 사용자와 음식점 집합 구하기
    users = df[user_col].unique()
    businesses = df[item_col].unique()

    # 사용자별로 부정적인 조합 저장
    user_negative_samples = {}

    # 각 사용자에 대해 부정적인 조합 생성
    for user in users:
        # 사용자가 방문한 음식점 집합
        user_visited_businesses = set(df[df[user_col] == user][item_col].unique())

        # 사용자가 방문하지 않은 음식점 찾기
        user_not_visited_businesses = list(set(businesses) - user_visited_businesses)

        # 랜덤 샘플링하여 부정적인 조합 생성
        if len(user_not_visited_businesses) > n: #n개 이상이면 데이터가 많아지니 제한을 둠
            user_negative_samples[user] = random.sample(user_not_visited_businesses, n)
        else:
            user_negative_samples[user] = user_not_visited_businesses

    # 부정적인 조합을 CSV 파일로 저장
    with open(f"{output_dir}/negative.csv", 'w') as f:
        for user, negatives in user_negative_samples.items():
            f.write(f"{user},{','.join(map(str, negatives))}\n")

    print("파일 'negative.csv'가 생성되었습니다.")

generate_negative_samples(df, n=100, output_dir="/content/drive/MyDrive/BPR")


파일 'negative.csv'가 생성되었습니다.


### 테스트셋의 미방문 아이템셋 생성

In [ ]:
#(u,i,j)형태 트리플 셋을 만들기 위한 코드
import pandas as pd
import random

# 개인별로 방문하지 않은 음식점 샘플링
def generate_individual_negative_sample(df, user_id, n=100, user_col='newUserId', item_col='newBusinessId'):
    # 모든 음식점 목록 가져오기
    businesses = df[item_col].unique()
    # 사용자가 방문한 음식점 목록 가져오기
    user_visited_businesses = set(df[df[user_col] == user_id][item_col].unique())
    # 사용자가 방문하지 않은 음식점 목록 계산
    user_not_visited_businesses = list(set(businesses) - user_visited_businesses)
    # 방문하지 않은 음식점 중에서 n개를 랜덤으로 샘플링
    if len(user_not_visited_businesses) > n:
        random_negative = random.sample(user_not_visited_businesses, n)
    else:
        random_negative = user_not_visited_businesses
    return random_negative

# 파일 경로 설정
path = "/content/drive/MyDrive/BPR/"

# testData를 위한 부정적인 음식점 샘플링
with open(path + 'negative.csv', 'r') as negative_file, open(path + 'test.csv', 'r') as test_file, open(path + 'test_negative.csv', 'w') as test_negative_file:
    read_negative = negative_file.readlines()
    read_test = test_file.readlines()
    print(len(read_negative))  # 12062 -> 사용자 수만큼 개인별로 방문하지 않은 음식점 집합 완성

    count=0
    for i in range(len(read_negative)):  # 각 사용자에 대해
        negative = read_negative[i]  # 사용자 i에 대한 방문하지 않은 음식점 집합 가져오기
        user_id, user_id_negative_businesses = negative.strip().split(',', 1)  # 사용자 ID와 방문하지 않은 음식점 구분
        #user_id_negative_businesses = user_id_negative_businesses.split(',')  # 방문하지 않은 음식점들을 리스트로 분리

        for j in range(i * 3, (i + 1) * 3):  # 각 사용자의 3개의 음식점 방문기록
            test = read_test[j % len(read_test)]  # 테스트 데이터 중 해당 사용자의 음식점 방문기록
            user_trip = test.strip()  # 공백 제거


            # 개별 사용자의 부정적인 음식점 생성
            individual_negative_sample = generate_individual_negative_sample(df, int(user_id), n=100)

            # 테스트 데이터와 부정적인 샘플을 결합하여 git에 있는 BPR모델 입력에 맞게 변환
            test_negative_business = user_trip + '\t' + '\t'.join(map(str, individual_negative_sample)) + '\n'
            if(count<3):
              print(test_negative_business)
              count+=1
            test_negative_file.write(test_negative_business)

print("파일 'test_negative.csv'가 생성되었습니다.")




12062
1	1	5663	998	4591	2630	1909	2458	3660	4774	6267	5456	5784	1896	5865	1551	2427	2063	5031	2111	1466	2943	3889	1030	1698	1847	6117	1506	2007	1722	763	1928	723	5882	4712	6011	1046	5705	197	6275	3996	5383	2357	6993	2654	499	1949	5821	2179	1384	1401	6657	2130	5984	2615	5355	4982	5249	2102	2101	5443	5219	2784	2953	903	6567	3745	1660	2070	3522	511	6215	1821	4786	5373	117	5923	5075	4980	3701	3740	2025	1890	1336	1757	4895	2435	216	1502	1036	6502	5054	2187	6351	3277	4106	1147	1795	2837	1612	4298	2578

1	2	3303	2371	6714	2566	3144	1247	3716	4532	5970	3243	2315	4905	6784	2443	5040	3494	4708	1231	2088	2865	4608	2450	2825	4910	6813	6124	1539	5139	6140	6935	6333	6745	5341	4529	4045	6005	5651	4251	1364	4614	6359	6296	6585	1282	2406	145	1992	57	4222	2238	5879	1865	752	4666	1504	5431	1975	6056	3447	65	157	3536	1684	6783	3580	6766	1709	2318	4928	4877	2000	6594	1503	1091	6529	5611	1721	3744	2232	3403	4679	1160	6019	4191	3663	6696	2515	6720	2333	4950	1254	2236	953	998	5962	3790	892	6160	5517	77

1	3	4